## Notebook to run eQTL analysis

#### import libraries and set notebook variables

In [1]:
import pandas as pd
import random
import os
import numpy as np
import threading
import dask.dataframe as dd
import statsmodels.stats.multitest as smm

In [ ]:
# parameter variables
cohort = 'ppmi'
amp_abbr = 'PP'
version = 'amppdv1'
tissue = 'wb'
visit = 0
visit_name = 'BLM0T1'

In [2]:
# naming
cohort_version = f'{cohort}.{version}'
cohort_build = f'{cohort}.{tissue}{visit}'

# directories
wrk_dir = f'/labshare/raph/eqtl/{cohort}'
geno_dir = f'{wrk_dir}/genotypes'
expr_dir = f'{wrk_dir}/expression'
info_dir = f'{wrk_dir}/sample_info'
plink_out_dir = f'{wrk_dir}/plink/output'
results_dir = f'{wrk_dir}/results'

# input files
gencode_pkl = '/labshare/raph/datasets/amppd/expression/gencode.v29.primary_assembly.annotation.pkl'
expr_pheno_file = '{exprdir}/{cohortbuild}.chr{chr}.pheno'
pfiles = '{genodir}/{cohortversion}.chr{chr}'
genome_index_file = '/labseq/Genomes/GRCh38_hg38/broad.v0/Homo_sapiens_assembly38.fasta.fai'

# output files
eqtl_psam = f'{info_dir}/{cohort_build}.psam'
chr_detected_out_file = '{exprdir}/{cohortbuild}.detected.genes.chr{chr}'
plink_glm_out_dir = f'{plink_out_dir}/glm'

# constant values
autosomes = [str(x) for x in list(range(1,23))]
max_dist = 1000000
capture_out = !(nproc)
max_threads = int(capture_out[0])
alpha_value = 0.05

#### threading related functions

In [3]:
# run command line procsss with bash majic
# make this a small function so it can be target function for threading
def run_bash_cmd(this_cmd):
    !{this_cmd}

# for each bash command is list add to execution thread, join and wait til all done
def run_bash_cmds_threaded(cmd_list):
    job_threads = []
    for this_cmd in cmd_list:
        this_thread = threading.Thread(target=run_bash_cmd, args=(this_cmd,))
        job_threads.append(this_thread)
        this_thread.start()

    for job_thread in job_threads:
        job_thread.join()          

#### check random autosome and see how many samples are in both genotypes and expression files

In [4]:
chrom = random.sample(autosomes, 1)[0]
# read the psam
psam_df = pd.read_csv(pfiles.format(genodir=geno_dir,
                                    cohortversion=cohort_version, chr=chrom) + '.psam',
                      sep='\s+')
print(f'chrom {chrom} psam shape is {psam_df.shape}')
expr_df = pd.read_csv(expr_pheno_file.format(exprdir=expr_dir,
                                             cohortbuild=cohort_build, chr=chrom),
                      sep='\s+')
print(f'chrom {chrom} expr shape is {expr_df.shape}')

keep_psam_df = psam_df.loc[psam_df['IID'].isin(expr_df['IID'])]
print(f'number of samples intersecting between geno and expr is {keep_psam_df.shape[0]}')

# save the psam for those samples that are present
keep_psam_df.to_csv(eqtl_psam, index=False, sep='\t')

chrom 10 psam shape is (1381, 4)
chrom 10 expr shape is (1512, 620)
number of samples intersecting between geno and expr is 1291


#### load the gencode annotations

In [5]:
gencode_df = pd.read_pickle(gencode_pkl)
print(gencode_df.shape)
chr_lengths_df = pd.read_csv(genome_index_file, header=None, sep='\t')
print(chr_lengths_df.shape)

(2742734, 91)
(3366, 5)


#### format the plink2 glm jobs

In [6]:
def subset_gene_annots(gene_ids, gencode_df, expr_dir, cohort_build, chrom):
    chr_genes_df = gencode_df.loc[gencode_df['gene_id'].isin(gene_ids)].copy()
    # compute the +/- cis proximal start and stop
    chr_genes_df['cisstart'] = chr_genes_df['start'] - max_dist
    chr_genes_df['cisstop'] = chr_genes_df['end'] + max_dist    
    # clean up if cis start or end is beyond chromosome beginning or end
    chr_genes_df.loc[chr_genes_df['cisstart'] < 0, 'cisstart'] = 0
    max_end = chr_lengths_df.loc[chr_lengths_df[0] == 'chr{}'.format(chrom),1].values[0]
    chr_genes_df.loc[chr_genes_df['cisstop'] > max_end, 'cisstop'] = max_end
    chr_genes_df = chr_genes_df[['gene_id', 'transcript_id','seqname', 'start',
                                 'end', 'strand', 'cisstart', 'cisstop', 'gene_name']]
    chr_genes_df.drop_duplicates(keep='first', inplace=True)
    return chr_genes_df


def largest_cis_gene_boundry(gene_id, genes_df):
    this_gene_df = genes_df.loc[genes_df['gene_id'] == gene_id]
    min_start = this_gene_df['cisstart'].min()
    max_stop =  this_gene_df['cisstop'].max()
    return min_start, max_stop
    

def frmt_plink2_glm_cmd(pfile, pheno_file, chrom, start_bp, end_bp, pheno_name,
                        samples_file, out_dir, name, min_mac = 5, max_miss_rate = 0.05):
    this_cmd = f'plink2 --silent --pfile {pfile} \
--mac {min_mac} --geno {max_miss_rate} --glm cols=+a1count,+a1freq \
--pheno {pheno_file} --pheno-name {pheno_name} \
--chr chr{chrom} --from-bp {start_bp} --to-bp {end_bp} \
--keep {samples_file} --out {out_dir}/{name}'
    return this_cmd

In [7]:
%%time

plink_glm_cmds = []
for chrom in autosomes:
    # load the pheno file
    pheno_file = expr_pheno_file.format(exprdir=expr_dir,
                                        cohortbuild=cohort_build, chr=chrom)
    expr_df = pd.read_csv(pheno_file, sep='\s+')
    # get list of gene IDs preset, drop the sample ID cols
    chr_gene_ids_list = list(set(expr_df.columns) - set({'FID', 'IID'}))
    chr_genes_df = subset_gene_annots(chr_gene_ids_list, gencode_df, expr_dir,
                                      cohort_build, chrom)
    this_file = chr_detected_out_file.format(exprdir=expr_dir,
                                             cohortbuild=cohort_build,
                                             chr=chrom)
    chr_genes_df.to_csv(this_file, sep='\t', index=False, header=True)
    
    # setup the output dir for running the glm per trait
    chr_out_dir = f'{plink_glm_out_dir}/chr{chrom}'
    os.makedirs(chr_out_dir, exist_ok=True)
    # iterate over genes in chrom to format plink2 glm tests
    for gene_id in chr_gene_ids_list:
        cisstart, cisstop = largest_cis_gene_boundry(gene_id, chr_genes_df)
        pfile = pfiles.format(genodir=geno_dir, 
                              cohortversion=cohort_version, chr=chrom)
        this_cmd = frmt_plink2_glm_cmd(pfile, pheno_file, chrom, cisstart, 
                                       cisstop, gene_id, eqtl_psam,
                                       chr_out_dir, cohort_build)
        plink_glm_cmds.append(this_cmd)

CPU times: user 2min, sys: 1.72 s, total: 2min 2s
Wall time: 2min 2s


#### run the glm jobs threaded

In [8]:
print(len(plink_glm_cmds))
# split the command list into max proc bins
glm_cmds_partioned = np.array_split(plink_glm_cmds, 
                                    np.floor(len(plink_glm_cmds)/max_threads/2))
print(len(glm_cmds_partioned))

16169
336


In [9]:
%%time

# interate over the batches of jobs running each sub-batch in parallel
for index, this_list in enumerate(glm_cmds_partioned):
    print(index, end=' ')
    run_bash_cmds_threaded(this_list.tolist())

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 Error: No variants remaining after main filters.
250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 2

#### rough check of output file counts

In [14]:
!ls -lh {plink_glm_out_dir}/chr*/{cohort_build}.*.glm.linear | wc -l

16163


#### accumulate results by chromosome

In [11]:
# small function to parse in plink2 glm linear results files
def read_chrom_glm_linear_results(in_dir, chrom, name, 
                                  min_mac = 5, min_freq = 0.01):
    results_files = f'{in_dir}/chr{chrom}/{name}.*.glm.linear'
    # specify the col dtypes to try and conserver mem
    col_dtypes = {'#CHROM':'int64', 'POS':'int64', 'ID':'object', 'REF':'object',
                  'ALT':'object', 'A1':'object', 'A1_CT':'int64', 
                  'A1_FREQ':'float64', 'OBS_CT':'int64', 
                  'BETA':'float64', 'SE':'float64', 'T_STAT':'float64',
                  'P':'float64'}
    results_dd = dd.read_csv(results_files, sep='\s+',
                             dtype=col_dtypes, usecols=col_dtypes.keys(),
                             include_path_column=True)
    # drop variants not appropriate for analysis
    results_dd = results_dd.loc[(results_dd['A1_CT'] >= min_mac) & 
                                (results_dd['A1_FREQ'] >= min_freq)]
    # trim included file path
    file_prefix = f'{in_dir}/chr{chrom}/{name}.'
    file_type = '.glm.linear'
    results_dd['gene_id'] = results_dd['path'].str.replace(file_prefix,'').str.replace(file_type,'')
    # drop the path column on return
    results_dd = results_dd.drop(columns=['path'])
    return results_dd.compute()


def compute_fdr(pvalues):
    bh_adj = smm.fdrcorrection(pvalues)
    return bh_adj[1]


def accumulate_chrom_results(expr_dir, cohort_build, chrom, plink_glm_out_dir):
    chrom_results_df = read_chrom_glm_linear_results(plink_glm_out_dir, chrom,
                                                     cohort_build)
            
    # add the corrected p-value, note just based on all chrom gene pvalues    
    chrom_results_df['bh_fdr'] = compute_fdr(chrom_results_df['P'].fillna(1))
    
    chrom_results_file = f'{plink_glm_out_dir}/{cohort_build}.chr{chrom}.glm.hdf5'
    chrom_results_df.to_hdf(chrom_results_file, key='glm_results', mode='w')
    print(f'chr{chrom} {chrom_results_df.shape}')

In [ ]:
%%time

# accumulate the chromsome results in parallel
job_threads = []
for chrom in autosomes:    
    this_thread = threading.Thread(target=accumulate_chrom_results,
                                   args=(expr_dir, cohort_build, chrom, plink_glm_out_dir))
    job_threads.append(this_thread)
    this_thread.start()

for job_thread in job_threads:
    job_thread.join()     

#### accumulate the statictically significant results

In [15]:
%%time

# accumulate the statictically significant results into single file
os.makedirs(results_dir, exist_ok=True)
results_df = None
for chrom in autosomes:
    chrom_results_file = f'{plink_glm_out_dir}/{cohort_build}.chr{chrom}.glm.hdf5'
    this_df = pd.read_hdf(chrom_results_file)
    print(f'chr{chrom} {this_df.shape}', end=' ')
    # subset the statictically significant results
    this_df = this_df.loc[this_df['bh_fdr'] <= alpha_value]
    print(this_df.shape, end=' ')
    results_df = pd.concat([results_df, this_df])
    
print(results_df.shape)

chr1 (9009280, 15) (221873, 15) chr2 (6785142, 15) (213562, 15) chr3 (5649024, 15) (150585, 15) chr4 (4314373, 15) (147095, 15) chr5 (4528274, 15) (123351, 15) chr6 (7344353, 15) (470054, 15) chr7 (5323211, 15) (201300, 15) chr8 (3814521, 15) (113187, 15) chr9 (3849236, 15) (82274, 15) chr10 (3965972, 15) (124794, 15) chr11 (5231319, 15) (127785, 15) chr12 (5765674, 15) (188490, 15) chr13 (2121558, 15) (53770, 15) chr14 (4470382, 15) (74202, 15) chr15 (3475585, 15) (100465, 15) chr16 (4611951, 15) (99742, 15) chr17 (5824002, 15) (195911, 15) chr18 (1928191, 15) (30211, 15) chr19 (7075705, 15) (149389, 15) chr20 (2527513, 15) (51014, 15) chr21 (1139873, 15) (37096, 15) chr22 (2316754, 15) (52133, 15) (3008283, 15)
CPU times: user 1min 15s, sys: 43 s, total: 1min 58s
Wall time: 2min 29s


In [16]:
sig_gene_cnt = len(results_df['gene_id'].unique())
print(f'total number of genes with eQTL {sig_gene_cnt}')

total number of genes with eQTL 15135


#### save the statictically significant results

In [17]:
%%time
results_df.to_hdf(f'{results_dir}/{cohort_build}.eqtl.hdf5', 
                  key='glm_results', mode='w')

CPU times: user 3.19 s, sys: 930 ms, total: 4.12 s
Wall time: 4.8 s


#### clean up the individual gene glm files

In [18]:
for chrom in autosomes:
    !rm {plink_glm_out_dir}/chr{chrom}/{cohort_build}.*
    
!ls -lh {plink_glm_out_dir}/chr*/{cohort_build}.* | wc -l

ls: cannot access /labshare/raph/eqtl/ppmi/plink/output/glm/chr*/ppmi.wb0.*: No such file or directory
0
